In [243]:
experiment_name = "vision"
results_dir = './results/' + experiment_name + '/'

In [244]:
import seaborn as sns
sns.set_theme(style="ticks")
sns.set_context("poster")

import matplotlib.pyplot as plt
import pandas as pd
import pickle 
import os 

import numpy as np
import pandas
    
import runs.config_experiments_vision as run
experiment_list = run.config_experiments(results_dir, create_json=False)

4705 config files created


In [245]:
name_attacks = ["bound", "l1_pgd_norm", "l1_fgm_norm", "linf_pgd", "l2_pgd_norm", "linf_fgsm", "l2_fgm_norm"]

In [247]:

list_entries = []
for net in ['ThreeLayer']:
    for dataset_id,dataset_name in zip([0,66,67],['MNIST','Fashion', 'CIFAR']):

        #Find an experiment of the dataset_id to get the epsilons
        for idx,_ in enumerate(experiment_list[1:]):
            if experiment_list[idx+1]['data_set'] == dataset_id: 
                break
        
        for attack in name_attacks:

            file_name = results_dir + experiment_list[idx+1]['model_name'] + '/results/acc_' + 'val' + '_' + attack + '.pkl'
            if not os.path.isfile(file_name):
                print("Missing!! " + file_name)
                continue
            with open(file_name, 'rb') as f:
                tmp = pickle.load(f)
                


            for idx_epsilon, cv_epsilon in enumerate(list(tmp.keys())):
                # Hash table of parameters
                parameters = {"epsilon": {}, "standarize": {}, "backbone": {}, "initial_learning_rate": {}, 
                              "robust_training": {}, "type_robust": {}, "epsilon_pgd_training":{}}

                to_exclude = [0]
                experiment_list_tmp = [element for i, element in enumerate(experiment_list) if i not in to_exclude]
                for exp in experiment_list_tmp:
                    if not exp['data_set'] == dataset_id: 
                        continue
                    for kk in parameters.keys():
                        if exp[kk] in parameters[kk]:
                            parameters[kk][exp[kk]].append(int(exp["model_name"]))
                        else:
                            parameters[kk][exp[kk]] = [int(exp["model_name"])]


                # For all methods, do cross-val and create an entry of the results
                backbones = [net, net+"+pgd"]



                for backbone in backbones:
                    for robust_training in [True, False]:
                        if robust_training:
                            type_robust_trainings = ['linf','l1',"certificate",'grad']
                        else:
                            type_robust_trainings = ['none']
                        for type_robust in type_robust_trainings:

                            if (backbone == 'Madry' and robust_training == True) or \
                                (backbone == 'CNN+clipping' and robust_training == False):
                                continue

                            if robust_training==False:
                                ids = list(set(parameters["backbone"][backbone]) & 
                                           set(parameters["robust_training"][False]))
                            else:
                                ids = list(set(parameters["backbone"][backbone]) & 
                                       set(parameters["robust_training"][True])&
                                      set(parameters["type_robust"][type_robust]))



                            if backbone == net + '+pgd' and robust_training == True:
                                continue

                            if ids == []:
                                continue
                            #print(ids)


                            # Cross-validation among learning rates and epsilons:
                            best_acc = -1
                            best_id = ids[0]
                            for id in ids:
                                if ((robust_training == False) & (backbone==net) \
                                    & (experiment_list[id]['training_batch_size']==256)):
                                    continue
                                    
                                file_name = results_dir + experiment_list[id]['model_name'] + '/results/acc_' + 'val' + '_' + attack + '.pkl'
                                if not os.path.isfile(file_name):
                                    print("Missing!! " + file_name)
                                    continue

                                with open(file_name, 'rb') as f:
                                    tmp_acc = pickle.load(f)

                                acc = list(tmp_acc.values())[idx_epsilon] #tmp_acc[cv_epsilon]
                                if acc>best_acc:
                                    best_id = id
                                    best_acc = acc

                            if best_acc == -1:
                                continue

                            if (robust_training == False) & (backbone==net):  
                                name_legend = 'vanilla'
                            elif backbone== net + '+pgd':
                                name_legend = 'pgd'
                            else:
                                if type_robust=='certificate':
                                    name_legend = 'RUB'
                                elif type_robust=='linf':
                                    name_legend = 'aRUB_Linf'
                                elif type_robust=='grad':
                                    name_legend = 'grad'
                                else:
                                    name_legend = 'aRUB_L1'

                            entry = {"learning_rate": experiment_list[best_id]['initial_learning_rate'],
                                     "net": net,
                                     "dataset": dataset_name,
                                     "standarize": experiment_list[best_id]['standarize'],
                                    "robust_training": name_legend,
                                    "epsilon": experiment_list[best_id]['epsilon'],
                                    "epsilon_pgd_training": experiment_list[best_id]['epsilon_pgd_training']}

                            dataset = "test"
                            entry["attack"] = attack
                            entry["experiment_id"] = best_id

                            with open(results_dir + experiment_list[best_id]['model_name'] + '/results/acc_' + dataset + '_' + 
                                attack + '.pkl', 'rb') as f:
                                tmp_acc = pickle.load(f)

                            entry["accuracy"] =  100*(list(tmp_acc.values())[idx_epsilon]) #tmp_acc[cv_epsilon]
                            entry["test_epsilon"] = cv_epsilon

                            list_entries.append(entry.copy())

df_results = pd.DataFrame.from_dict(list_entries) 


